# Creation of in vitro insertion parts for pfa chain in C.Cinerea

Now that promoters have been collected and generated and ran through a quantative flouresence analysis we can create the full list of inserts and necesarry primers for the top 5 primers and terminators. 

This insert will contain: <br>
- PABA marker gene <br>
- PUFA synthesis pathway <br>
    - pfa1,2,3 and a pptase <br>
- Hygromicin marker gene. 

*this is where we insert a cool image of the creation*

## Loading Libraries

## Genetic elements

## Primer creation using TEEMI